# MISP

In [1]:
%pip install pymisp


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [3]:
from dotenv import dotenv_values

In [4]:
from surianalytics.connectors import getGitRoot, check_str_bool

In [5]:
import os
import shutil

In [6]:
config = dotenv_values(os.path.join(getGitRoot(), ".env") if shutil.which("git") is not None else os.path.join(os.path.expanduser("~"), ".env"))

In [7]:
from pymisp import PyMISP

In [8]:
misp = PyMISP(config["MISP_HOST"],
              config["MISP_TOKEN"],
              check_str_bool(config["MISP_TLS_VERIFY"]),
              debug=False)

In [16]:
publish_timestamp="1d"

In [17]:
import pandas as pd

In [18]:
attributes = [
    "url",
    "domain",
    "ip-dst",
    "ip-dst|port"
]

In [12]:
DF_ATTR = pd.DataFrame()
for attr in attributes:
    resp = pd.json_normalize(misp.search(controller='attributes', 
                                                    publish_timestamp=publish_timestamp,
                                                    timestamp="1d"
                                                    type_attribute=attr, 
                                                    category="Network activity", 
                                                    pythonify=False).get("Attribute"))
    DF_ATTR = pd.concat([DF_ATTR, resp], axis=0)
    print(attr, len(resp))

url 0
domain 0
ip-dst 0
ip-dst|port 0


In [13]:
DF_ATTR.columns.values

array([], dtype=int64)

In [14]:
DF_ATTR.head(3)

""


In [15]:
DF_ATTR.groupby("type").agg({"value": "nunique",
                             "event_id": "nunique"})

KeyError: 'type'

In [ ]:
DF_ATTR.groupby("Event.info").agg({"type": ["unique", "nunique"],
                                   "value": ["unique", "nunique"]})

# Scan for IoC values via scirius

## Domains

## IP addrs